# Introduction
In this laboratory we will get our hands dirty working with Large Language Models (e.g. GPT and BERT) to do various useful things. I you haven't already, it is highly recommended to:

+ Read the [Attention is All you Need](https://arxiv.org/abs/1706.03762) paper, which is the basis for all transformer-based LLMs.
+ Watch (and potentially *code along*) with this [Andrej Karpathy video](https://www.youtube.com/watch?v=kCc8FmEb1nY) which shows you how to build an autoregressive GPT model from the ground up.

# Exercise 1: Warming Up
In this first exercise you will train a *small* autoregressive GPT model for character generation (the one used by Karpathy in his video) to generate text in the style of Dante Aligheri. Use [this file](https://archive.org/stream/ladivinacommedia00997gut/1ddcd09.txt), which contains the entire text of Dante's Inferno (**note**: you will have to delete some introductory text at the top of the file before training). Train the model for a few epochs, monitor the loss, and generate some text at the end of training. Qualitatively evaluate the results 

In [3]:
# See gpt.py code [ Code copyied from Karpathy ]

from gpt import *

model = GPTLanguageModel()
model = model.to(device)
model.load_state_dict(torch.load("/home/hsilva/DLA/Lab2/Models/nanogpt_4500.pt"))

decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(model.generate(context, max_new_tokens=500)[0].tolist()))






'al con ostre tanto ben quelle furo
  ch'a ben la rimana appura; bandommi queschi

m'io dise: <Perch'io parla tui non scponda
  ch'i' potrei qui tra doppila quella,
  o susciaremo di che 'n la dotta,
  ch'elli hiserta fe' l'a natto casca>>.

Sa gia` per matta da partien del Bacio
  canta in su lo scoperto, ancor tu mi piace
  perche' sospensava tanto molto,

per consie in cie; e giustizzar li arsi
  se 'l falcun la tua via officio;
  quel l'angoscia no che tanto conento,

per non quel che posa 


# Exercise 2: Working with Real LLMs

Our toy GPT can only take us so far. In this exercise we will see how to use the [Hugging Face](https://huggingface.co/) model and dataset ecosystem to access a *huge* variety of pre-trained transformer models.

## Exercise 2.1: Installation and text tokenization

First things first, we need to install the [Hugging Face transformer library](https://huggingface.co/docs/transformers/index):

    conda install -c huggingface -c conda-forge transformers
    
The key classes that you will work with are `GPT2Tokenizer` to encode text into sub-word tokens, and the `GPT2LMHeadModel`. **Note** the `LMHead` part of the class name -- this is the version of the GPT2 architecture that has the text prediction heads attached to the final hidden layer representations (i.e. what we need to **generate** text). 

Instantiate the `GPT2Tokenizer` and experiment with encoding text into integer tokens. Compare the length of input with the encoded sequence length.

**Tip**: Pass the `return_tensors='pt'` argument to the togenizer to get Pytorch tensors as output (instead of lists).

In [ ]:
import torch
from transformers import GPT2LMHeadModel, GPT2TokenizerFast
tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")
sequence = "I enjoy DLA lectures but"
inputs = tokenizer(sequence, return_tensors="pt")

print(f"Token length:{len(inputs['input_ids'][0])}, Sequence length: {len(sequence)}")

Token length:6, Sequence length: 24


## Exercise 2.2: Generating Text

There are a lot of ways we can, given a *prompt* in input, sample text from a GPT2 model. Instantiate a pre-trained `GPT2LMHeadModel` and use the [`generate()`](https://huggingface.co/docs/transformers/v4.27.2/en/main_classes/text_generation#transformers.GenerationMixin.generate) method to generate text from a prompt.

**Note**: The default inference mode for GPT2 is *greedy* which might not results in satisfying generated text. Look at the `do_sample` and `temperature` parameters.

In [48]:
greedy_output = model.generate(inputs['input_ids'], max_length=50, pad_token_id=50256)
print(tokenizer.decode(greedy_output[0], skip_special_tokens=True))

I enjoy DLA lectures but I don't like to spend time with my friends. I'm not a big fan of the "I'm not a big fan of the "I'm not a big fan of the "I'm not a big fan


In [50]:
greedy_output = model.generate(inputs['input_ids'], max_length=50, pad_token_id=50256,do_sample=True,temperature=0.1)
print(tokenizer.decode(greedy_output[0], skip_special_tokens=True))

I enjoy DLA lectures but I don't like to spend time with people who are not my friends. I don't like to spend time with people who are not my friends. I don't like to spend time with people who are not my friends


In [49]:
greedy_output = model.generate(inputs['input_ids'], max_length=50, pad_token_id=50256,do_sample=True,temperature=0.9)
print(tokenizer.decode(greedy_output[0], skip_special_tokens=True))

I enjoy DLA lectures but don't really study the book by itself. Also, I read a lot of books that were on'mystical' topics, and that's not great for my academic career, but I'm not really into esoteric


In [52]:
beam_output = model.generate(inputs['input_ids'],max_length=50,num_beams=5,no_repeat_ngram_size=2,early_stopping=True, pad_token_id=50256)
print(tokenizer.decode(beam_output[0], skip_special_tokens=True))

I enjoy DLA lectures but I don't think I've ever been able to get to the point where I feel like I'm in the right place."

He added: "I think it's important for people to be aware of what they


In [51]:
beam_output = model.generate(inputs['input_ids'],max_length=50,num_beams=5,no_repeat_ngram_size=2,early_stopping=True, pad_token_id=50256, do_sample=True,temperature=0.9)
print(tokenizer.decode(beam_output[0], skip_special_tokens=True))

I enjoy DLA lectures but I don't think I've ever been in a class like this before."

"I think you're right," she said. "I'm not sure I'm going to be able to do it again. I


# Exercise 3: Reusing Pre-trained LLMs (choose one)

Choose **one** of the following exercises (well, *at least* one). In each of these you are asked to adapt a pre-trained LLM (`GPT2Model` or `DistillBERT` are two good choices) to a new Natural Language Understanding task. A few comments:

+ Since GPT2 is a *autoregressive* model, there is no latent space aggregation at the last transformer layer (you get the same number of tokens out that you give in input). To use a pre-trained model for a classification or retrieval task, you should aggregate these tokens somehow (or opportunistically select *one* to use).

+ BERT models (including DistillBERT) have a special [CLS] token prepended to each latent representation in output from a self-attention block. You can directly use this as a representation for classification (or retrieval).

+ The first *two* exercises below can probably be done *without* any fine-tuning -- that is, just training a shallow MLP to classify or represent with the appropriate loss function.

# Exercise 3.1: Training a Text Classifier (easy)

Peruse the [text classification datasets on Hugging Face](https://huggingface.co/datasets?task_categories=task_categories:text-classification&sort=downloads). Choose a *moderately* sized dataset and use a LLM to train a classifier to solve the problem.

**Note**: A good first baseline for this problem is certainly to use an LLM *exclusively* as a feature extractor and then train a shallow model.

# Exercise 3.2: Training a Question Answering Model (harder)

Peruse the [multiple choice question answering datasets on Hugging Face](https://huggingface.co/datasets?task_categories=task_categories:multiple-choice&sort=downloads). Chose a *moderately* sized one and train a model to answer contextualized multiple-choice questions. You *might* be able to avoid fine-tuning by training a simple model to *rank* the multiple choices (see margin ranking loss in Pytorch).

# Exercise 3.3: Training a Retrieval Model (hardest)

The Hugging Face dataset repository contains a large number of ["text retrieval" problems](https://huggingface.co/datasets?task_categories=task_categories:text-retrieval&p=1&sort=downloads). These tasks generally require that the model measure *similarity* between text in some metric space -- naively, just a cosine similarity between [CLS] tokens can get you pretty far. Find an interesting retrieval problem and train a model (starting from a pre-trained LLM of course) to solve it.

**Tip**: Sometimes identifying the *retrieval* problems in these datasets can be half the challenge. [This dataset](https://huggingface.co/datasets/BeIR/scifact) might be a good starting point.